# Library import

In [1]:
import openpyxl as op
import pandas as pd
import numpy as np
import win32com.client as win32

# TEA functions

In [32]:
def InputValue(filename, target_data):
    
    # TEA file open
    wb = op.load_workbook(filename)
    capex_opex = wb['Plastic']
    
    # CAPEX
    cells = [f'F{jj}' for jj in range(5,9)] + [f'F{jj}' for jj in range(10,15)]
    blocks_input = ['HDPE[kg/hr]', 'LDPE[kg/hr]', 'PP[kg/hr]', 'PS[kg/hr]',
               'RPLUGIN', 'RPLUGIN2', 'RPYROIN', 'RPYROOUT', 'S1']     # RPLUGIN, RPYROOUT ==> 열교환면적 데이터로 수정
    
    for _blck, _cell in zip(blocks_input, cells):
        capex_opex[_cell] = target_data[_blck]
        
    # OPEX
    cells = [f'C{jj}' for jj in range(23,27)] + [f'C{jj}' for jj in range(28,32)]
    block_hd = ['HDPDCOM_HD', 'LPDECOM_HD', 'PPDECOM_HD', 'PSDECOM_HD',
              'HEATER_HD', 'RPLUG_HD', 'RPYRO_HD', 'B2']
    
    for _hd, _cell in zip(block_hd, cells):
        capex_opex[_cell] = target_data[_hd]
    
    wb.save(filename=filename)
    wb.close()

def SaveFile(filename):
    excel = win32.Dispatch("Excel.Application") 
    wb = excel.Workbooks.Open(r"C:\Users\PSID_PC22\Documents\GitHub\TeamEnv\TEA\{}".format(filename))
    excel.Visible = False
    wb.Save()
    excel.Quit()

def LoadResults(filename):
    re_load = op.load_workbook(filename, data_only=True)
    tea_res = re_load['Plastic EAC&TPC']
    eac = tea_res['E34'].value
    tpc = tea_res['K25'].value
    return eac, tpc
    

# Load aspen data

In [ ]:
data = pd.read_csv('Results_pyrolysis_python.csv')
data.columns

# Analyse economics

In [38]:
tea_file = 'TEA_plastic_pyro_v2.xlsx'

tea_res = []
for ii in range(len(data)):
    target_data = data.iloc[ii,:]
    InputValue(tea_file, target_data)
    SaveFile(tea_file)
    EAC, TPC = LoadResults(tea_file)
    tea_res.append([EAC, TPC])

# Save results

In [41]:
data[['EAC', 'TPC']] = tea_res
data.to_csv('Results_TEA_pyrolysis.csv')